In [5]:
class Example:
    a = 3
    def f(self, p):
        """A function that takes a single argument"""
        return f'The parameter was {p}'

In [8]:
e = Example()
print(f'Instance {e}')
print(f'Method   {e.f}')
print(f'Calling the method with 3 returns {repr(e.f(3))}')

Instance <__main__.Example object at 0x1031aa550>
Method   <bound method Example.f of <__main__.Example object at 0x1031aa550>>
Calling the method with 3 returns 'The parameter was 3'


In [11]:
print(dir(Example))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'a', 'f']


In [12]:
print(dir(e))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'a', 'f']


In [13]:
dir(Example) == dir(e)

True

In [14]:
print(e.a)
e.a += 1
print(e.a)

3
4


In [16]:
e.b = 4
print(e.b)

4


In [17]:
def double(a):
    return a + a

print(double(1))
print(double('x'))

2
xx


In [18]:
by_hand = type('ByHand', (), {'f': Example.f, 'a': 3})

In [19]:
print(type(by_hand))

<class 'type'>


In [20]:
bh = by_hand()
print(bh)

In [21]:
print(bh.f('fred'))
print(bh.a)

The parameter was fred
3


In [22]:
bh.b = 3
print(bh.b)

3


In [23]:
o = object()
print(dir(o))

['__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__']


In [24]:
o.a = 1

AttributeError: 'object' object has no attribute 'a'

In [43]:
def by_hand_hand():
    return by_hand()

bh2 = by_hand_hand()
print(bh2.a)

3


In [44]:
print(dir(by_hand_hand))

['__annotations__', '__call__', '__class__', '__closure__', '__code__', '__defaults__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__get__', '__getattribute__', '__globals__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__kwdefaults__', '__le__', '__lt__', '__module__', '__name__', '__ne__', '__new__', '__qualname__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__']


In [50]:
def class_by_hand():
    def f(self, arg):
        return f'f called with {self!r} and {arg!r}'
    cls = type('ByHand', (), {})
    cls.a = 3
    cls.f = f
    return cls

c = class_by_hand()
print(f'Class {c!r}')
print(f'Class value a {c.a!r}')
print(f'Class function {c.f(None, "fred")}')
o = c()
print(f'Instance {o!r}')
print(f'Instance vaue a {o.a!r}')
print(f'Instance function {o.f("fred")}')

Class <class '__main__.ByHand'>
Class value a 3
Class function f called with None and 'fred'
Instance <__main__.ByHand object at 0x1031ddb50>
Instance vaue a 3
Instance function f called with <__main__.ByHand object at 0x1031ddb50> and 'fred'


In [51]:
class_by_hand.b = 3

In [52]:
print(class_by_hand.b)

3


In [53]:
type(class_by_hand)

function

In [54]:
help(class_by_hand)

Help on function class_by_hand in module __main__:

class_by_hand()



In [56]:
class_by_hand.__doc__ = 'A function that returns a class it made'
help(class_by_hand)

Help on function class_by_hand in module __main__:

class_by_hand()
    A function that returns a class it made



In [57]:
print(class_by_hand.__dict__)

{'b': 3}


In [111]:
def fn():
    pass

In [112]:
def bare_f(self, arg):
    return f'f called with {self!r} and {arg!r}'
print(type(bare_f))

<class 'function'>


In [113]:
fn.f = bare_f
fn.a = 3
print(f'Calling f {fn.f(None, "fred")}')
print(f'Value a {fn.a}')

Calling f f called with None and 'fred'
Value a 3


In [114]:
fn.f(by_hand2, 'fred')

"f called with <function by_hand2 at 0x103198310> and 'fred'"

In [115]:
class_by_hand.__call__()

__main__.ByHand

In [116]:
fn.__call__()

In [117]:
print(fn.__code__)

<code object fn at 0x1031e4be0, file "/var/folders/3c/74720f_907b07qy3vwcklsg00000gp/T/ipykernel_24937/3060542578.py", line 1>


In [118]:
import dis
dis.dis(fn)

  2           0 LOAD_CONST               0 (None)
              2 RETURN_VALUE


In [119]:
dis.dis(fn.__code__)

  2           0 LOAD_CONST               0 (None)
              2 RETURN_VALUE


In [120]:
dis.dis(bare_f)

  2           0 LOAD_CONST               1 ('f called with ')
              2 LOAD_FAST                0 (self)
              4 FORMAT_VALUE             2 (repr)
              6 LOAD_CONST               2 (' and ')
              8 LOAD_FAST                1 (arg)
             10 FORMAT_VALUE             2 (repr)
             12 BUILD_STRING             4
             14 RETURN_VALUE


In [121]:
print(dir(fn))

['__annotations__', '__call__', '__class__', '__closure__', '__code__', '__defaults__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__get__', '__getattribute__', '__globals__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__kwdefaults__', '__le__', '__lt__', '__module__', '__name__', '__ne__', '__new__', '__qualname__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'a', 'f']


In [122]:
print(fn.__dict__)

{'f': <function bare_f at 0x103198f70>, 'a': 3}


In [123]:
print(fn.__call__)

<method-wrapper '__call__' of function object at 0x1031ec940>


In [124]:
print(fn.__call__())

None


In [125]:
print(dir(fn.f))

['__annotations__', '__call__', '__class__', '__closure__', '__code__', '__defaults__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__get__', '__getattribute__', '__globals__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__kwdefaults__', '__le__', '__lt__', '__module__', '__name__', '__ne__', '__new__', '__qualname__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__']


In [126]:
save = fn.__code__
fn.__code__ = bare_f.__code__
print(fn(None, 'fred'))
fn.__code__ = save
print(fn())

f called with None and 'fred'
None
